## machine_temperature_system_failure

In [7]:
#  LOADER

import pandas as pd
import json

def load_nab_file(csv_path):
    """ Lecture CSV """
    df = pd.read_csv(csv_path)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    return df.set_index('timestamp')

def add_label_column(df, labels_path, file_key):
    df = df.sort_index()

    with open(labels_path, 'r') as f:
        all_labels = json.load(f)
    
    windows = all_labels.get(file_key, [])
    df['anomaly'] = 0 
    
    for start, end in windows:
        start_dt = pd.to_datetime(start)
        end_dt = pd.to_datetime(end)
        
        df.loc[start_dt:end_dt, 'anomaly'] = 1 
        
    return df, windows

# Visu

import plotly.graph_objects as go

def plot_anomalies(df, labels, file_key):
    fig = go.Figure()

    # Donnée réelle (Value)
    fig.add_trace(go.Scatter(
        x=df.index, y=df['value'],
        line=dict(color='#636EFA', width=1.5),
        name='Température'
    ))

    # Fenêtres d'anomalies réelles (NAB Labels)
    for i, (start, end) in enumerate(labels):
        fig.add_vrect(
            x0=start, x1=end,
            fillcolor="red", opacity=0.2,
            layer="below", line_width=0,
            annotation_text="Panne réelle" if i == 0 else "",
            annotation_position="top left"
        )

    fig.update_layout(
        template='plotly_dark',
        title=f"Analyse : {file_key}",
        xaxis_title="Date/Heure",
        yaxis_title="Valeur",
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        hovermode="x unified"
    )

    fig.show()

# df

csv_path = "data/raw/realKnownCause/machine_temperature_system_failure.csv" # DATA
labels_path = "data/labels/combined_windows.json" # LABELS
file_key = "realKnownCause/machine_temperature_system_failure.csv" # CLE DU LABELS

df = load_nab_file(csv_path)
df, labels = add_label_column(df, labels_path, file_key)

plot_anomalies(df, labels, file_key)

In [11]:
df.head()

,value,anomaly
timestamp,,
2013-12-02 21:15:00,73.967322,0
2013-12-02 21:20:00,74.935882,0
2013-12-02 21:25:00,76.124162,0
2013-12-02 21:30:00,78.140707,0
2013-12-02 21:35:00,79.329836,0


## EXPORT STREAMLIT

In [14]:
# Créaton csv Streamlit

import pandas as pd

def to_streamlit(df_iqr, df_iso_forest, df_LSTMEncoder, df_prophet, df_lof, df):

    final_df = df.loc[:, ~df.columns.duplicated()].copy()
    final_df = final_df[['timestamp', 'value', 'anomaly']]
    final_df = final_df.rename(columns={'anomaly': 'actual_anomaly'})

    # 2. Modèle 1 (IQR)
    final_df = final_df.merge(df_iqr[['timestamp', 'yhat', 'upper', 'lower']], on='timestamp', how='left')
    final_df = final_df.rename(columns={'yhat': 'yhat_i', 'upper': 'up_i', 'lower': 'lo_i'})
    
    # Modèle 2 (Isolation Forest)
    final_df = final_df.merge(df_iso_forest[['timestamp', 'yhat', 'upper', 'lower']], on='timestamp', how='left')
    final_df = final_df.rename(columns={'yhat': 'yhat_if', 'upper': 'up_if', 'lower': 'lo_if'})
    
    # Modèle 3 (LSTM)
    final_df = final_df.merge(df_LSTMEncoder[['timestamp', 'yhat', 'upper', 'lower']], on='timestamp', how='left')
    final_df = final_df.rename(columns={'yhat': 'yhat_ae', 'upper': 'up_ae', 'lower': 'lo_ae'})

    # Modèle 4 (Prophet)
    final_df = final_df.merge(df_prophet[['timestamp', 'yhat', 'upper', 'lower']], on='timestamp', how='left')
    final_df = final_df.rename(columns={'yhat': 'yhat_p', 'upper': 'up_p', 'lower': 'lo_p'})

    # Modèle 5 (LOF)
    final_df = final_df.merge(df_lof[['timestamp', 'yhat', 'upper', 'lower']], on='timestamp', how='left')
    final_df = final_df.rename(columns={'yhat': 'yhat_l', 'upper': 'up_l', 'lower': 'lo_l'})

    # Calcul des détections individuelles
    final_df['anom_i'] = ((final_df['value'] > final_df['up_i']) | (final_df['value'] < final_df['lo_i'])).astype(int)
    final_df['anom_if'] = ((final_df['value'] > final_df['up_if']) | (final_df['value'] < final_df['lo_if'])).astype(int)
    final_df['anom_ae'] = ((final_df['value'] > final_df['up_ae']) | (final_df['value'] < final_df['lo_ae'])).astype(int)
    final_df['anom_p'] = ((final_df['value'] > final_df['up_p']) | (final_df['value'] < final_df['lo_p'])).astype(int)
    final_df['anom_l'] = ((final_df['value'] > final_df['up_l']) | (final_df['value'] < final_df['lo_l'])).astype(int)

    # LOGIQUE DE VOTE PONDEREE
    final_df['vote_score'] = (0.8*final_df['anom_i']) + (1.2*final_df['anom_if']) + (1.2*final_df['anom_ae']) + final_df['anom_p'] + (0.8*final_df['anom_l'])
    
    # alert_level pour les couleurs
    final_df['alert_level'] = final_df['vote_score'].apply(lambda x: 2 if x >= 3 else (1 if x >= 1 else 0))

    final_df['is_anomaly'] = (final_df['alert_level'] == 2).astype(int)

    final_df.to_csv("data_results.csv", index=False)

df_iqr = pd.read_csv("df_iqr.csv")
df_iso_forest = pd.read_csv("df_iso_forest.csv")
df_LSTMEncoder = pd.read_csv("df_LSTMEncoder.csv")
df_prophet = pd.read_csv("df_prophet.csv")
df_lof = pd.read_csv("df_lof.csv")

for d in [df_iqr, df_iso_forest, df_LSTMEncoder, df_prophet, df_lof]:
    d['timestamp'] = pd.to_datetime(d['timestamp'])

df = df.reset_index()
df['timestamp'] = pd.to_datetime(df['timestamp'])

to_streamlit(df_iqr, df_iso_forest, df_LSTMEncoder, df_prophet, df_lof, df)